In [1]:
from glob import glob

In [2]:
import numpy as np
np.set_printoptions(suppress=True)
from numpy import eye, dot
import cv2 as cv

In [3]:
import chessboard
import park_martin

In [4]:
from scipy.linalg import expm, inv

In [5]:
img_names = glob('./data/images/image??.png')
img_list = [cv.imread(img_name, 0) for img_name in img_names]

In [6]:
pose_names = glob('./data/poses/pose??.txt')

In [7]:
rob_pose_list = [np.loadtxt(pose_name) for pose_name in pose_names]

In [8]:
camera_matrix, dist_coeffs = chessboard.calibrate_lens(img_list)

In [9]:
def hat(v):
    return [[   0, -v[2],  v[1]],
            [v[2],     0, -v[0]],
            [-v[1],  v[0],    0]]

In [10]:
def tf_mat(r, t):
    res = eye(4)
    res[0:3, 0:3] = expm(hat(r))
    res[0:3, -1] = t
    return res

In [11]:

obj_pose_list = []
corner_list = []
for i, img in enumerate(img_list):
    found, corners = chessboard.find_corners(img)
    corner_list.append(corners)
    if not found:
        raise Exception("Failed to find corners in img # %d" % i)
    rvec, tvec = chessboard.get_object_pose(
        chessboard.pattern_points, corners, camera_matrix, dist_coeffs)
    object_pose = tf_mat(rvec, tvec)
    obj_pose_list.append(object_pose)

In [12]:
A, B = [], []
for i in range(1,len(img_list)):
    p = rob_pose_list[i-1], obj_pose_list[i-1]
    n = rob_pose_list[i], obj_pose_list[i]
    A.append(dot(inv(p[0]), n[0]))
    B.append(dot(inv(p[1]), n[1]))

In [13]:
# Transformation to chessboard in robot gripper
X = eye(4)
Rx, tx = park_martin.calibrate(A, B)
X[0:3, 0:3] = Rx
X[0:3, -1] = tx

print("X: ")
print(X)

X: 
[[ 0.14745783 -0.55154848  0.82100576  0.04066306]
 [ 0.86928682  0.46823208  0.15842708 -0.06329481]
 [-0.47180145  0.69032818  0.54849831  0.751672  ]
 [ 0.          0.          0.          1.        ]]


In [14]:
print("For validation. Printing transformations from the robot base to the camera")
print("All the transformations should be quite similar")

for i in range(len(img_list)):
    rob = rob_pose_list[i]
    obj = obj_pose_list[i]
    tmp = dot(rob, dot(X, inv(obj)))
    print(tmp)

# Here I just pick one, but maybe some average can be used instead
rob = rob_pose_list[0]
obj = obj_pose_list[0]
cam_pose = dot(dot(rob, X), inv(obj))

For validation. Printing transformations from the robot base to the camera
All the transformations should be quite similar
[[-0.24523428 -0.76656934 -0.59349106 -0.1529003 ]
 [ 0.9017303  -0.40517059  0.15072862 -0.34338586]
 [-0.35600908 -0.49820506  0.7906006   0.26441113]
 [ 0.          0.          0.          1.        ]]
[[ 0.16037487 -0.89339663 -0.41966948 -0.33598279]
 [ 0.93870769  0.26948373 -0.21495669 -0.46417796]
 [ 0.30513564 -0.35947329  0.88185666 -0.03592851]
 [ 0.          0.          0.          1.        ]]
[[-0.20716406 -0.97646958  0.05991695 -0.7063825 ]
 [ 0.75008667 -0.19785713 -0.6310488  -0.0251003 ]
 [ 0.62805499 -0.08578771  0.77342577  0.02302095]
 [ 0.          0.          0.          1.        ]]
[[-0.73671601 -0.58192419 -0.34440343 -0.3651727 ]
 [ 0.67582614 -0.61666204 -0.40371646 -0.40687071]
 [ 0.02255189 -0.53018119  0.8475844  -0.30562148]
 [ 0.          0.          0.          1.        ]]
[[ 0.12517234 -0.84695969 -0.51671182 -0.3438272 ]
 [ 0.8

# Hand-eye Calibration

Resource: 
- http://campar.in.tum.de/Chair/HandEyeCalibration 

## Eye-in-hand

![](https://www.torsteinmyhre.name/snippets/robcam_calibration/images/extrinsic-camera-calibration-eye-in-hand-651px.png)

## Eye-to-hand

![](https://www.torsteinmyhre.name/snippets/robcam_calibration/images/extrinsic-camera-calibration-stationary-camera-651px.png)